In [82]:
import pandas as pd

def process_and_merge_games(file_paths):

    processed_dfs = []
    
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        
        # 'Source' column and filter for "tickets"
        df['Source'] = df.apply(lambda row: "tickets" if row['Type'] == 'Ticket' and pd.isna(row['STRefID']) else "members", axis=1)
        df = df[df['Source'] == 'tickets']
        
        # remove duplicates and drop rows with missing key columns
        df = df.drop_duplicates(subset=['assign using  ID number'])
        df = df.dropna(subset=['assign using  ID number', 'First name', 'Last name', 'Email', 'Phone No.'])
        
        # relevant columns
        df = df[['assign using  ID number', 'First name', 'Last name', 'Email', 'Phone No.']]
        
        # append the processed DataFrame to the list
        processed_dfs.append(df)
    
    # merge
    final_df = processed_dfs[0]
    for i, df in enumerate(processed_dfs[1:], start=1):
        final_df = pd.merge(
            final_df, 
            df, 
            on='assign using  ID number', 
            how='inner',  # find common attendees
            suffixes=(f'', f'_game{i+1}')
        )
    
    # drop unnecessary columns dynamically
    columns_to_drop = [col for col in final_df.columns if '_game' in col or col == 'Event name']
    final_df = final_df.drop(columns=columns_to_drop)
    
    # rename columns ending with '_game1' to their original names
    final_df.columns = [col.replace('_game1', '') if col.endswith('_game1') else col for col in final_df.columns]
    
    return final_df

file_paths = [
    'AuthorizedAttendanceReportCsv_🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת"א.csv',
    'AuthorizedAttendanceReportCsv_Round 2_ Hapoel Tel Aviv 🚗.csv'
]

final_df = process_and_merge_games(file_paths)

final_df.to_excel('big_games_attendees.xlsx')

final_df

assign using  ID number First name  Last name                      Email  \
0                  43467760        עדו   מידן דוד         idomedan@gmail.com   
1                 218340206       יאיר  פויכטונגר       yayafe2009@gmail.com   
2                 318591658        גיא        כהן      cguymichael@gmail.com   
3                 217286798        אור   מילביצקי        ormil2008@gmail.com   
4                 217527126       יואב      סמואל      yoavsamuel8@gmail.com   
..                      ...        ...        ...                        ...   
150               217285725     נימרוד    כהן צחי       nimidrive2@gmail.com   
151               032963068      איילת      אהרון    aharon.ayelet@gmail.com   
152               213188410      אילון    גרינבלט  aylongreenblatt@gmail.com   
153               335046181       יובל    בן שלוש         giladnan@gmail.com   
154               217643311       אורי      אהרון        hrwnwry35@gmail.com   

       Phone No.  
0     0544997453  
1     0503026309  
2     0508230306  
3    05152572013  
4     0585907569  
..           ...  
150   0508876333  
151   0525523268  
152   0544745106  
153   0548887144  
154   0538211433  

[155 rows x 5 columns]